In [ ]:
import pandas as pd

from aust_covid.inputs import get_ifrs, load_pop_data
from emutools.tex import DummyTexDoc
from aust_covid.model import build_model
from inputs.constants import SUPPLEMENT_PATH
from emutools.parameters import load_param_info

In [ ]:
param_info = load_param_info()
param_info['value'].update(get_ifrs(DummyTexDoc()))
parameters = param_info['value'].to_dict()
epi_model = build_model(DummyTexDoc())

In [ ]:
init_pop = epi_model.get_initial_population(parameters=parameters)
model_pops = load_pop_data(DummyTexDoc())
states = epi_model.stratifications['states'].strata
for age in epi_model.stratifications['agegroup'].strata:
    for state in states:
        model_pops.loc[int(age), f'check_{state}'] = init_pop[init_pop.index.str.contains(f'states_{state}') & init_pop.index.str.contains(f'agegroup_{age}X')].sum()

imm_check = {}
for imm in epi_model.stratifications['immunity'].strata:
    imm_check[imm] = init_pop[init_pop.index.str.contains(f'immunity_{imm}')].sum()

print([v / sum(imm_check.values()) for v in imm_check.values()])
model_pops.plot()